In [59]:
from ddl import load_project
import pandas as pd
from collections import defaultdict


In [18]:
vdj, _ = load_project({'vdj_path': '/Users/nick/ddl_website/instance/uploads/homotypic/processed_vdj.h5ddl',
                    'adata_path': 'NULL'})

In [108]:
def merge_data(vdj):
    
    def merge_dicts(lst):
        dct = defaultdict(list)
        for d in lst:
            for k, v in d.items():
                dct[k].append(v)
        return dict(dct)

    data = vdj.data
    metadata = vdj.metadata
    data['locus_dict'] = data.apply(lambda row: {row['locus']: row['sequence_alignment_aa']}, axis=1)
    data = data[['locus_dict']].reset_index()
    data['sequence_id'] = data['sequence_id'].str.replace(r'_contig_[12]', '', regex=True)
    data = data.groupby('sequence_id').agg(list).reset_index()
    data['locus_dict'] = data['locus_dict'].apply(merge_dicts)
    locus = pd.json_normalize(data['locus_dict'])
    data= pd.concat([data.drop(columns=['locus_dict']), locus], axis=1)
    data['IGK'] = data['IGK'].apply(lambda lst: ','.join(lst) if isinstance(lst, list) else lst)
    data['IGH'] = data['IGH'].apply(lambda lst: ','.join(lst) if isinstance(lst, list) else lst)
    data['IGL'] = data['IGL'].apply(lambda lst: ','.join(lst) if isinstance(lst, list) else lst)
    merged = metadata.merge(data, left_on=metadata.index, right_on=data['sequence_id'])
    merged.drop(['key_0'], axis=1, inplace=True)
    merged.set_index('sequence_id', inplace=True)
    
    return merged

    

In [109]:
merge_data(vdj)

,clone_id,clone_id_by_size,locus_VDJ,locus_VJ,productive_VDJ,productive_VJ,v_call_VDJ,d_call_VDJ,j_call_VDJ,v_call_VJ,j_call_VJ,c_call_VDJ,c_call_VJ,junction_VDJ,junction_VJ,junction_aa_VDJ,junction_aa_VJ,v_call_B_VDJ,d_call_B_VDJ,j_call_B_VDJ,v_call_B_VJ,j_call_B_VJ,c_call_B_VDJ,c_call_B_VJ,productive_B_VDJ,productive_B_VJ,umi_count_B_VDJ,umi_count_B_VJ,v_call_VDJ_main,v_call_VJ_main,d_call_VDJ_main,j_call_VDJ_main,j_call_VJ_main,c_call_VDJ_main,c_call_VJ_main,v_call_B_VDJ_main,d_call_B_VDJ_main,j_call_B_VDJ_main,v_call_B_VJ_main,j_call_B_VJ_main,isotype,isotype_status,locus_status,chain_status,rearrangement_status_VDJ,rearrangement_status_VJ,clone_id_size,IGK,IGH,IGL
sequence_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
/Users/nick/ddl_website/instance/uploads/homotypic/homotypic_AAACCGGAGCCGCTAG,B_VDJ_124_1_1_VJ_32_1_3,255,IGH,IGK,T,T,"IGHV5-6-1,IGHV5-6",None,IGHJ3,IGKV3-4,IGKJ1,"IGHG1,IGHG3",IGKC,TGTGCAAGACAGAGGTATCCCTGGTTTGCTTACTGG,TGTCAGCAAAGTAATGAGGATCCGTGGACGTTC,CARQRYPWFAYW,CQQSNEDPWTF,"IGHV5-6-1,IGHV5-6",None,IGHJ3,IGKV3-4,IGKJ1,"IGHG1,IGHG3",IGKC,T,T,8.0,12.0,"IGHV5-6-1,IGHV5-6",IGKV3-4,None,IGHJ3,IGKJ1,"IGHG1,IGHG3",IGKC,"IGHV5-6-1,IGHV5-6",None,IGHJ3,IGKV3-4,IGKJ1,IgG,IgG,IGH + IGK,Single pair,standard,standard,1.0,DIVLTQSPTSLAVSLGQRATISCKASQSVAYDGNVYMNWYQQKPGQ...,EVQLVESGGDLVKPGGSLKLSCAASGFTFSSYGMSWVRQTPDKRLE...,NaN
/Users/nick/ddl_website/instance/uploads/homotypic/homotypic_AAACCGTGTCATCTAT,B_VDJ_79_2_14_VJ_151_1_4|B_VDJ_79_2_14_VJ_97_1_27,23|545,IGH,IGK|IGK,T,T|T,IGHV1-80,None,IGHJ4,IGKV4-72|IGKV14-111,IGKJ1|IGKJ2,None,IGKC|IGKC,TGTGCAAGATCGACTTTTTATCCTATGGACTACTGG,TGCCAGCAGTGGAGTAGAAACCCGTGGACGTTC|TGTCTACAGTAT...,CARSTFYPMDYW,CQQWSRNPWTF|CLQYAEFPYTF,IGHV1-80,None,IGHJ4,IGKV4-72|IGKV14-111,IGKJ1|IGKJ2,None,IGKC|IGKC,T,T|T,4.0,85.0,IGHV1-80,IGKV4-72,None,IGHJ4,IGKJ1,None,IGKC,IGHV1-80,None,IGHJ4,IGKV4-72,IGKJ1,None,None,IGH + Extra VJ,Extra pair,standard,standard,9.0,QIVLSQSPAILSASPGEKVTMTCRASSSVNYMHWYQQKPGSSPKPW...,NaN,NaN
/Users/nick/ddl_website/instance/uploads/homotypic/homotypic_AAACCTGTCAATCCCT,B_VDJ_115_1_8_VJ_128_1_5,39,IGH,IGK,T,T,IGHV1-42,"IGHD2-9,IGHD2-4",IGHJ2,IGKV12-41,IGKJ4,"IGHG1,IGHG3",IGKC,TGTGCAAGATCTGAGGATTACGACAGTGTCGGCTACTTTGACTACTGG,TGTCAACATTTTTGGAGTTCTCCATTCACGTTC,CARSEDYDSVGYFDYW,CQHFWSSPFTF,IGHV1-42,"IGHD2-9,IGHD2-4",IGHJ2,IGKV12-41,IGKJ4,"IGHG1,IGHG3",IGKC,T,T,15.0,66.0,IGHV1-42,IGKV12-41,"IGHD2-9,IGHD2-4",IGHJ2,IGKJ4,"IGHG1,IGHG3",IGKC,IGHV1-42,"IGHD2-9,IGHD2-4",IGHJ2,IGKV12-41,IGKJ4,IgG,IgG,IGH + IGK,Single pair,standard,standard,7.0,DIQMTQSPASLSASVGETVTITCRPSGNIHIYLAWYHQKQGKSPQL...,EVQLQQSGPELVKPGASVKISCKASGYSFTGYYMNWVKQSPEKSLE...,NaN
/Users/nick/ddl_website/instance/uploads/homotypic/homotypic_AAACCTGTCATGACCA,B_VDJ_59_1_8_VJ_30_1_4,3,IGH,IGK,T,T,IGHV2-6,None,IGHJ1,IGKV5-43,IGKJ2,"IGHG1,IGHG3",IGKC,TGTGCCAGACATGGGAATGATGTTTATTCCCACTTATACTTCGATG...,TGTCAACAGAGTAACAGCTGGCCCACGTTC,CARHGNDVYSHLYFDVW,CQQSNSWPTF,IGHV2-6,None,IGHJ1,IGKV5-43,IGKJ2,"IGHG1,IGHG3",IGKC,T,T,14.0,10.0,IGHV2-6,IGKV5-43,None,IGHJ1,IGKJ2,"IGHG1,IGHG3",IGKC,IGHV2-6,None,IGHJ1,IGKV5-43,IGKJ2,IgG,IgG,IGH + IGK,Single pair,standard,standard,30.0,DIVLTQSPATLSVTPGDSVSLSCRASQSISNNLHWYQQKSHESPRL...,QVQLKESGPGLVAPSQSLSITCTVSGFSLTTYGVHWVRQPPGKGLE...,NaN
/Users/nick/ddl_website/instance/uploads/homotypic/homotypic_AAACGATCAAATGGCA,B_VDJ_115_1_8_VJ_128_1_6,1,IGH,IGK,T,T,IGHV1-42,"IGHD2-9,IGHD2-4",IGHJ2,IGKV12-41,IGKJ4,"IGHG1,IGHG3",IGKC,TGTGCAAGATCTGAGGATTACGACAGTGTCGGCTACTTTGACTACTGG,TGTCAACATTTTTGGAGTACTCCATTCACGTTC,CARSEDYDSVGYFDYW,CQHFWSTPFTF,IGHV1-42,"IGHD2-9,IGHD2-4",IGHJ2,IGKV12-41,IGKJ4,"IGHG1,IGHG3",IGKC,T,T,28.0,110.0,IGHV1-42,IGKV12-41,"IGHD2-9,IGHD2-4",IGHJ2,IGKJ4,"IGHG1,IGHG3",IGKC,IGHV1-42,"IGHD2-9,IGHD2-4",IGHJ2,IGKV12-41,IGKJ4,IgG,IgG,IGH + IGK,Single pair,standard,standard,174.0,DIQMTQSPASLSASVGETVTITCRASGNIHIYLAWYQQKQGKSPQL...,EVQLQQSGPELVKPGASVKISCKASGYSFTGHYMNWVKQSPEKSLE...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.